In [1]:
import sys
sys.path.append('../src')
sys.path.append('../utils')
import time
from onhand_sqls import *
from email_tools import *
from atp_tools import Ending_On_Hand
import warnings
warnings.filterwarnings("ignore")


In [2]:
t1 = time.time()
eoh = Ending_On_Hand()
time.time() - t1

0.017998218536376953

In [3]:
def insert_pd_tosql(tablename, df, writer = False): #functional block: insert
    if writer == True:
        df['Writer'] = writer
    quoted = urllib.parse.quote_plus('''
                                    DRIVER={ODBC Driver 17 for SQL Server};
                                    SERVER=qcells-us-atp-db-server.database.windows.net;
                                    DATABASE=us-qcells-atp-db;
                                    UID=qcells;
                                    Pwd={Asdqwe123!@#}''')
    engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted),  fast_executemany=True)
    print('[EVENT] INSERT INFO: \n', '\t TABLE NAME: ', tablename, '\n', '\t TABLE COLUMNS ARE BELOW', df.columns)
    chucksize = math.floor(2100/len(df.columns)) - 1
    df.to_sql(tablename, con=engine, if_exists='append', method='multi', index=False, chunksize=chucksize)
    engine.dispose()

In [4]:
t5 = time.time()
df_ending_onhand_result_join = eoh.Ending_onhand_main()
print(time.time() -t5)

AZURE DB CONNECTION RE-TRYING NOW COUNT: 0 Times
AZURE DB CONNECTION RE-TRYING NOW COUNT: 1 Times
115.73361611366272


In [5]:
t6 = time.time()
eoh.check_batch_is_updated('ATP_BI_ENDING_ONHAND')
print(time.time() -t6)

[EVENT] ATP_BI_ENDING_ONHAND HAS ALREADY SAVED TODAY. RPA IS INITIALIZING NOW
31.027379512786865


In [6]:
len(df_ending_onhand_result_join)

119080

In [7]:
t7 = time.time()
insert_pd_tosql('ATP_BI_ENDING_ONHAND', df_ending_onhand_result_join)
print(time.time() - t7)

[EVENT] INSERT INFO: 
 	 TABLE NAME:  ATP_BI_ENDING_ONHAND 
 	 TABLE COLUMNS ARE BELOW Index(['WH_Location', 'Item_Code', 'ProductName', 'Power_Class', 'Tariff',
       'WestEast', 'Dates', 'CPO_shipped', 'Hard_Allocation', 'Intake',
       'ON_HAND', 'Soft_Allocation', 'CW_Type', 'SegmentName', 'YYYY_BI'],
      dtype='object')
173.4263837337494


In [8]:
from pbixapi_tools import * 

rbwa = Refresh_pbix_web_api()
rbwa.request_refresh()

Dataset refreshed


In [14]:
df_ending_onhand_result_join

,WH_Location,Item_Code,ProductName,Power_Class,Tariff,WestEast,Dates,CPO_shipped,Hard_Allocation,Intake,ON_HAND,Soft_Allocation,CW_Type,SegmentName,YYYY_BI
0,CA_N,99999973H,Q.PEAK DUO BLK ML G10+,400,0.1475,W,2022-09-30,0.0,0.0,0.0,0.0,0.0,Reconsil Period,Residential,2022
1,CA_N,99999973H,Q.PEAK DUO BLK ML G10+,400,0.1475,W,2022-10-01,0.0,0.0,0.0,0.0,0.0,Reconsil Period,Residential,2022
2,CA_N,99999973H,Q.PEAK DUO BLK ML G10+,400,0.1475,W,2022-10-02,0.0,0.0,0.0,0.0,0.0,Reconsil Period,Residential,2022
3,CA_N,99999973H,Q.PEAK DUO BLK ML G10+,400,0.1475,W,2022-10-03,0.0,0.0,0.0,0.0,0.0,Reconsil Period,Residential,2022
4,CA_N,99999973H,Q.PEAK DUO BLK ML G10+,400,0.1475,W,2022-10-04,0.0,0.0,0.0,0.0,0.0,Reconsil Period,Residential,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120430,GA_N,49409540,T400H,400,0.0000,E,2023-06-11,0.0,0.0,0.0,0.0,0.0,Future Week,Residential,2023
120431,GA_N,49409540,T400H,400,0.0000,E,2023-06-12,0.0,0.0,0.0,0.0,0.0,Future Week,Residential,2023
120432,GA_N,49409540,T400H,400,0.0000,E,2023-06-13,0.0,0.0,0.0,0.0,0.0,Future Week,Residential,2023
120433,GA_N,49409540,T400H,400,0.0000,E,2023-06-14,0.0,0.0,0.0,0.0,0.0,Future Week,Residential,2023
